In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [2]:
pip install streamlit matplotlib pillow


In [3]:
%pip install llama_index ftfy regex tqdm
%pip install git+https://github.com/openai/CLIP.git
%pip install torch torchvision
%pip install matplotlib scikit-image
%pip install -U qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the package

In [4]:
import os

os.chdir('/content/drive/MyDrive/GPT4_Vision_Product_Tagging/')

In [5]:
%%writefile app.py
import streamlit as st
import os
import time
import random, string
from pathlib import Path
from PIL import Image
import qdrant_client
from llama_index import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index import StorageContext
from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex
from llama_index.response.notebook_utils import display_source_node
from llama_index.schema import ImageNode
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.schema import ImageDocument

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
image_path = '/content/drive/MyDrive/GPT4_Vision_Product_Tagging/FashionImages'


def create_llm_index(client):
    #client = qdrant_client.QdrantClient(path = client)


    text_store = QdrantVectorStore(
        client=client, collection_name="text_collection"
    )
    image_store = QdrantVectorStore(
        client=client, collection_name="image_collection"
    )
    storage_context = StorageContext.from_defaults(vector_store=text_store)

    documents = SimpleDirectoryReader("/content/drive/MyDrive/GPT4_Vision_Product_Tagging/FashionImages/").load_data()
    st.write("creating index")
    index = MultiModalVectorStoreIndex.from_documents(
        documents, storage_context=storage_context, image_vector_store=image_store
    )

    return index

def retrieve_images(query,client):

    # Create Llama index
    index = create_llm_index(client)

    retriever = index.as_retriever(similarity_top_k=1, image_similarity_top_k=1)
    retrieval_results = retriever.retrieve(query)

    retrieved_images = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_images.append(res_node.node.metadata["file_path"])
    return retrieved_images


def image_retrieval(input_image_path,client):
    index = create_llm_index(client)

    retriever_engine = index.as_retriever(image_similarity_top_k=2)
    # retrieve more information from the GPT4V response

    retrieval_results = retriever_engine.image_to_image_retrieve(
        "./FashionImages/"+input_image_path
    )
    retrieved_images = []
    image_documents = [ImageDocument(image_path="./FashionImages/"+input_image_path)]

    for res in retrieval_results:
        retrieved_images.append(res.node.metadata["file_path"])

    for res_img in retrieved_images[1:]:
        image_documents.append(ImageDocument(image_path=res_img))

    openai_mm_llm = OpenAIMultiModal(
    model="gpt-4-vision-preview", api_key=os.environ["OPENAI_API_KEY"], max_new_tokens=1500
    )
    response = openai_mm_llm.complete(
        prompt="Given the first image as the base image, what the other images correspond to?",
        image_documents=image_documents,
    )

    return retrieved_images,response

# Streamlit app
def main():
    st.title("Multi-Modal Image Retrieval System")

    c1,c2 = st.tabs(["Search by text","Search by Image"])

    with c1:
      # Get user input query
      query = st.text_input("Enter a description to find matching images:")
      #existing_client = generate_random_name()
      client = qdrant_client.QdrantClient(":memory:")
      existing_client = client

      if st.button("Search"):
          # Retrieve images based on the query
          retrieved_images = retrieve_images(query,existing_client)

          # Display retrieved images
          if retrieved_images:
              st.subheader("Matching Images:")
              for img_path in retrieved_images:
                  st.image(img_path, caption=os.path.basename(img_path), use_column_width=True)
          else:
              st.warning("No matching images found.")
    with c2:
      # Upload an image
      st.title("Image Retrieval and Reasoning App")

    # Upload input image
      input_image = st.file_uploader("Upload Input Image", type=["jpg", "png"])
      if input_image:
          client = qdrant_client.QdrantClient(":memory:")
          existing_client = client
          st.image(input_image, caption="Uploaded Input Image.", use_column_width=True)
          input_image_path = input_image.name

          # Retrieve images based on the input image
          retrieved_images_paths = image_retrieval(input_image_path,existing_client)[0]

          # Display retrieved images
          st.subheader("Retrieved Images")
          for img_path in retrieved_images_paths:
              st.image(img_path, caption=img_path, use_column_width=True)


          # Display GPT-4V reasoning response
          st.subheader("GPT-4V Reasoning")
          reasoning_response = image_retrieval(input_image_path,existing_client)[1]
          st.write(reasoning_response)


if __name__ == "__main__":
    main()


Overwriting app.py


In [6]:

# Create a package.json file with default values
!echo '{"name": "colab", "version": "1.0.0", "main": "index.js", "scripts": {"start": "echo \"Error: no start script specified\" && exit 1"}, "keywords": [], "author": "", "license": "ISC"}' > /content/package.json

# Install localtunnel
!npm install localtunnel

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN colab@1.0.0 No description
npm WARN colab@1.0.0 No repository field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.61s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [8]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.74.141.149
npx: installed 22 in 3.165s
your url is: https://four-apples-lay.loca.lt
